In [29]:
import matplotlib.pyplot as plt
import json

def symmetry(shap, plot_name):
    """
    return value of symmetry for top/bot and left/right
    """
    check_list_top = [
        'lshoulder-lelbow', 'rshoulder-relbow', 'lelbow-lwrist', 'relbow-rwrist',
        'lshoulder', 'rshoulder', 'lelbow', 'relbow', 'lwrist', 'rwrist',
        'top_right', 'top_left']
    check_list_right = [
        'rshoulder-relbow', 'relbow-rwrist', 'rshoulder', 'relbow', 'rwrist',
        'rhip-rknee', 'rknee-rankle', 'rhip', 'rknee', 'rankle',
        'top_right', 'bot_right']

    top, bot, right, left = [], [], [], []
    nb_top, nb_bot, nb_right, nb_left = 0, 0, 0, 0

    for feature, shap_value in shap[plot_name].items():
        if feature == "Face":
            continue
        if plot_name[:11] == 'linear_data':
            feature = feature[:-2]
        if feature.lower() in check_list_top:
            top.append(shap_value)
            nb_top += 1
        else:
            bot.append(shap_value)
            nb_bot += 1
        if feature.lower() in check_list_right:
            right.append(shap_value)
            nb_right += 1
        else:
            left.append(shap_value)
            nb_left += 1

    top, bot, right, left = sum(top), sum(bot), sum(right), sum(left)

    if bot > top:
        sym_top_bot = round((bot / top), 2)
    else:
        sym_top_bot = round((top / bot), 2)
    if right > left:
        sym_left_right = round((right / left), 2) * -1
    else:
        sym_left_right = round((left / right), 2)

    return [sym_top_bot, sym_left_right]

In [30]:
# ouvrir shap dict
f = open('Data/V2/shap_v2.json')
shap = json.load(f)
f.close()

# ouvrir auc dict
f = open('Data/V2/auc_v2.json')
auc = json.load(f)
f.close()

In [31]:
datasets = [dataset for dataset in auc.keys()]

In [32]:
for dataset in datasets:
    t_b, l_r = [], []
    for label in ["_normal", "_anormal"]:
        auc[dataset]["mean"] = round(auc[dataset]["mean"], 2)
        auc[dataset]["std"] = round(auc[dataset]["std"], 2)
        auc[dataset]["sym" + label] = {}
        if dataset != "amplitudes":
            sym = symmetry(shap=shap, plot_name=dataset+label)
            auc[dataset]["sym" + label]["T-B"] = sym[0] 
            auc[dataset]["sym" + label]["L-R"] = sym[1] 
            t_b.append(sym[0])
            l_r.append(sym[1])
    auc[dataset]["sym_global"] = {
        "T-B": round(sum(t_b)/2, 2), 
        "L-R": round(sum(l_r)/2, 2)}

In [33]:
auc

{'angular_velocity': {'std': 0.12507623924165953,
  'mean': 0.6855720668220668,
  'sym_normal': {'T-B': 1.28, 'L-R': 1.53},
  'sym_anormal': {'T-B': 1.65, 'L-R': 1.78},
  'sym_global': {'T-B': 1.46, 'L-R': 1.66}},
 'linear_acceleration': {'std': 0.12722988606968638,
  'mean': 0.8888167388167387,
  'sym_normal': {'T-B': 2.41, 'L-R': 1.26},
  'sym_anormal': {'T-B': 1.67, 'L-R': -1.03},
  'sym_global': {'T-B': 2.04, 'L-R': 0.11}},
 'linear_velocity': {'std': 0.15977440840374593,
  'mean': 0.8964355089355089,
  'sym_normal': {'T-B': 1.57, 'L-R': -1.48},
  'sym_anormal': {'T-B': 1.12, 'L-R': -1.87},
  'sym_global': {'T-B': 1.35, 'L-R': -1.68}},
 'angular_jurk': {'std': 0.0938579213934219,
  'mean': 0.6625208125208124,
  'sym_normal': {'T-B': 1.91, 'L-R': -1.5},
  'sym_anormal': {'T-B': 1.14, 'L-R': 1.08},
  'sym_global': {'T-B': 1.52, 'L-R': -0.21}},
 'angular_data': {'std': 0.17497455402402012,
  'mean': 0.692074592074592,
  'sym_normal': {'T-B': 2.85, 'L-R': 2.74},
  'sym_anormal': {'T-B'